In [21]:
import gensim 
from os import walk

file_list = []
current_path = "../group_20/aclImdb/train/"

def read_input(current_path, file_list):
    for (dirpath, dirnames, _) in walk(current_path):
        for dirname in dirnames:
            for (_, _, filenames) in walk(current_path + dirname):
                for filename in filenames:
                    with open(current_path + dirname + '/' + filename, encoding='utf-8') as file:
                        for j, line in enumerate(file):
                            yield gensim.utils.simple_preprocess(line)

document = list(read_input(current_path, file_list))

Imdb_review_model = gensim.models.Word2Vec (document, size=50, window=10, min_count=2, workers=4)
Imdb_review_model.train(document, total_examples=len(document), epochs=10)

neg
pos
unsup


(129368659, 169992700)

In [22]:
Imdb_review_model.wv['fun']

array([-5.417965  , -0.29541355, -4.077036  , -0.15914145,  2.1616344 ,
        2.8794796 ,  2.0367963 ,  2.4561493 ,  1.0242066 ,  3.547608  ,
        1.2455571 , -0.0231355 ,  2.5833807 ,  4.6095834 , -0.6019455 ,
       -1.5181314 ,  0.35333687,  3.9718223 ,  0.93765205,  2.6573458 ,
       -3.5082197 ,  4.528874  , -0.7063457 ,  4.0898933 , -3.2253351 ,
        0.98315006,  0.484121  , -2.7076666 ,  0.46639752, -3.4604692 ,
        0.75192505, -7.180612  , -2.2534604 , -6.1422896 , -0.30873823,
       -1.0481647 ,  2.123767  ,  2.544564  , -2.7237117 ,  2.4106696 ,
       -1.9558036 ,  2.8908806 ,  1.0605178 , -0.8021128 , -4.104199  ,
        0.29835624, -1.9405675 ,  3.0352945 ,  4.140157  , -3.639733  ],
      dtype=float32)

In [23]:
Imdb_review_model.wv.most_similar(positive = ['fun'], topn=20)

[('entertaining', 0.7144640684127808),
 ('funny', 0.6970599889755249),
 ('enjoyable', 0.6909166574478149),
 ('laughs', 0.6376445293426514),
 ('laugh', 0.6105514764785767),
 ('cheesy', 0.6063879728317261),
 ('scary', 0.6060782074928284),
 ('hoot', 0.5967482924461365),
 ('campy', 0.5950718522071838),
 ('watchable', 0.5920515060424805),
 ('popcorn', 0.5835415124893188),
 ('good', 0.5758621692657471),
 ('entertainment', 0.5657311677932739),
 ('amusing', 0.5609129667282104),
 ('humor', 0.5581991076469421),
 ('exciting', 0.5539864897727966),
 ('worth', 0.5522226691246033),
 ('bearable', 0.5495377779006958),
 ('pleasant', 0.5490581393241882),
 ('hilarious', 0.5465127825737)]

In [66]:
import numpy as np

wordcat = ['entertaining', 'horror', 'cute', 'love', 'action']
v = []
for word in wordcat:
    v.append(Imdb_review_model.wv[word])
    
    
vocablist = []
for word, vect in Imdb_review_model.wv.vocab.items():
    vocablist.append(word)

file_list = []
current_path = "./"
sumdot = [0,0,0,0,0]
numsum = [0,0,0,0,0]
for (_, _, filenames) in walk(current_path):
    with open("1.txt", encoding='utf-8') as file:
        for j, line in enumerate(file):
            wordlist = gensim.utils.simple_preprocess(line)
            for word in wordlist:
                if word in vocablist:
                    for i in range(len(v)):
                        dotnum = np.dot(v[i], Imdb_review_model.wv[word]) / np.linalg.norm(v[i]) / np.linalg.norm(Imdb_review_model.wv[word])
                        #print(dotnum)
                        sumdot[i] += dotnum
                        #print(sumdot[i])
                        numsum[i] += 1
                        #print(numsum[i])
print(np.array(sumdot)/np.array(numsum))

[0.14256373 0.10643349 0.08020781 0.04523052 0.11495   ]
